In [1]:
import pandas as pd 
import numpy as np
from tqdm import tqdm
import os

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image


W, H, D = 224,224,3
path = 'test_dataset_test/test'
path_to_models = 'models/'
names2 = os.listdir(path)
names2 = [int(name.split('\\')[-1].split('.')[0]) for name in names2]
names2 = sorted(names2)
names = [path + f'\\{i}.jpg' for i in names2]

models = [load_model(path_to_models + name_model) for name_model in os.listdir(path_to_models)]
os.listdir(path_to_models)

['xception_1.h5', 'xception_2.h5', 'xception_3.h5']

In [2]:
test_imgs = []

for name in tqdm(names[0:]):
    img_path = os.path.join(path, name)
    img = image.load_img(img_path, target_size=(W, H, D))
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.
    test_imgs.append(img_tensor)

100%|██████████| 2138/2138 [00:05<00:00, 369.99it/s]


In [3]:
# class_names_all = ['truck_crane',
#                    'car', 
#                    'excavator',
#                    'human',
#                    'dump_truck',
#                    'mining_loader',
#                    'rink',
#                    'bulldozer']

# class_names_1 = ['car_human',
#                  'other']


# class_names_2 = ['car',
#                  'human']

# class_names_3 = ['truck_crane',
#                  'excavator',
#                  'dump_truck',
#                  'mining_loader',
#                  'rink',
#                  'bulldozer']

In [3]:
predicts = []

dict_2 = {0:1, 
          1:3}
dict_3 = {0:0,
          1:2,
          2:4,
          3:5,
          4:6,
          5:7}

for i, img in tqdm(enumerate(test_imgs), total=len(test_imgs)):
    pred = models[0].predict(img, verbose=0)
    if np.argmax(pred) == 0:
        pred2 = models[1].predict(img, verbose=0)
        predicts.append(dict_2[np.argmax(pred2)])
    else:
        pred3 = models[2].predict(img, verbose=0)
        predicts.append(dict_3[np.argmax(pred3)])              

100%|██████████| 2138/2138 [03:08<00:00, 11.36it/s]


In [4]:
PATH_TEST = r'F:\karches\test_dataset_test_\test.csv'
test_df = pd.read_csv(PATH_TEST)
test_df = test_df.drop(["class"], axis = 1)

test_df["class"] = predicts

test_df.head(30)

,ID_img,class
0,0.jpg,5
1,1.jpg,5
2,2.jpg,1
3,3.jpg,1
4,4.jpg,6
5,5.jpg,3
6,6.jpg,7
7,7.jpg,3
8,8.jpg,3
9,9.jpg,5


In [5]:
test_df.to_csv("submit.csv", index=False)